In [4]:
import tensorflow as tf
from tensorflow import keras

ما اینجا یک لایه کاستوم درست میکنیم که از لایه های کراس ارث بری داشته باشه 

هم به این صورت می توان نوشت که اصولی تره

In [3]:
class SimpleDense(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super().__init__(**kwargs)
        self.units = units  # تعداد نورون‌ها

    def build(self, input_shape):
        # اینجا دقیقاً input_shape[-1] را می‌دانیم
        self.w = self.add_weight(
            name="weight",
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True
        )
        self.b = self.add_weight(
            name="bias",
            shape=(self.units,),
            initializer="zeros",
            trainable=True
        )
        super().build(input_shape)  # حتماً بقیه‌ی build اصلی هم اجرا شود

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


هم به صورت زیر که وزن هارو ذخیره نمیکنه

In [52]:
class SimpleDense(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(SimpleDense, self).__init__(**kwargs)
        self.units = units
        self.w = None
        self.b = None

    def build(self, input_shape):
        input_dim = input_shape[-1]

        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, self.units), dtype="float32"),
            trainable=True,
            name="weight"
        )

        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(self.units,), dtype="float32"),
            trainable=True,
            name="bias"
        )

        # تیک زدن وضعیت ساخته‌شده لایه (اگر نیاز به ذخیره یا تحلیل داشته باشیم)
        super().build(input_shape)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


In [4]:
print(help(SimpleDense))

Help on class SimpleDense in module __main__:

class SimpleDense(keras.src.layers.layer.Layer)
 |  SimpleDense(units=32, **kwargs)
 |  
 |  Method resolution order:
 |      SimpleDense
 |      keras.src.layers.layer.Layer
 |      keras.src.backend.tensorflow.layer.TFLayer
 |      keras.src.backend.tensorflow.trackable.KerasAutoTrackable
 |      tensorflow.python.trackable.autotrackable.AutoTrackable
 |      tensorflow.python.trackable.base.Trackable
 |      keras.src.ops.operation.Operation
 |      keras.src.saving.keras_saveable.KerasSaveable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, units=32, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  build(self, input_shape)
 |  
 |  call(self, inputs)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from keras.src.layers.layer.Layer:
 |  
 |  __call__(self, *args, **kwargs)
 |      Call self as a function.
 |  
 |

In [30]:
dense_layer = SimpleDense(units=4)  # فقط units می‌دی
x = tf.ones((2, 2))                 # داده با سایز (batch_size=2, input_dim=2)
y = dense_layer(x)                 # اولین بار اجرا، خودش build رو صدا می‌زنه

print(dense_layer.weights)  # حالا وزن‌ها به‌درستی تعریف و ثبت شده‌اند


[<Variable path=simple_dense_2/weight, shape=(2, 4), dtype=float32, value=[[ 0.03005759  0.00363774 -0.00853379  0.06113647]
 [-0.06313009  0.01408493  0.02702316  0.02133872]]>, <Variable path=simple_dense_2/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>]


In [32]:
dense_layer.weights

[<Variable path=simple_dense_2/weight, shape=(2, 4), dtype=float32, value=[[ 0.03005759  0.00363774 -0.00853379  0.06113647]
  [-0.06313009  0.01408493  0.02702316  0.02133872]]>,
 <Variable path=simple_dense_2/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>]

In [34]:
dense_layer.trainable_weights

[<Variable path=simple_dense_2/weight, shape=(2, 4), dtype=float32, value=[[ 0.03005759  0.00363774 -0.00853379  0.06113647]
  [-0.06313009  0.01408493  0.02702316  0.02133872]]>,
 <Variable path=simple_dense_2/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>]

In [36]:
dense_layer.non_trainable_weights

[]

In [38]:
dense_layer.trainable

True

In [40]:
dense_layer.trainable = False
print(dense_layer.trainable_weights)  # باید خالی باشد
print(dense_layer.non_trainable_weights)  # باید وزن‌ها را نشان دهد

[]
[<Variable path=simple_dense_2/weight, shape=(2, 4), dtype=float32, value=[[ 0.03005759  0.00363774 -0.00853379  0.06113647]
 [-0.06313009  0.01408493  0.02702316  0.02133872]]>, <Variable path=simple_dense_2/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>]


In [42]:
dense_layer.trainable_weights

[]

In [44]:
dense_layer.non_trainable_weights

[<Variable path=simple_dense_2/weight, shape=(2, 4), dtype=float32, value=[[ 0.03005759  0.00363774 -0.00853379  0.06113647]
  [-0.06313009  0.01408493  0.02702316  0.02133872]]>,
 <Variable path=simple_dense_2/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>]

حالا امتحانش می کنیم و یه چیز الکی بهش میدیم

In [47]:
x = tf.ones((2, 2))
dense_layer = SimpleDense(units=4)
y = dense_layer(x)

# ببینیم وزن‌ها ثبت شده‌اند:
print("weights:", dense_layer.weights)
print("trainable_weights:", dense_layer.trainable_weights)
print("non_trainable_weights:", dense_layer.non_trainable_weights)


weights: [<Variable path=simple_dense_3/weight, shape=(2, 4), dtype=float32, value=[[-0.00958292  0.03199904  0.097617   -0.13530092]
 [ 0.00178712 -0.0261467  -0.03803866  0.10134279]]>, <Variable path=simple_dense_3/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>]
trainable_weights: [<Variable path=simple_dense_3/weight, shape=(2, 4), dtype=float32, value=[[-0.00958292  0.03199904  0.097617   -0.13530092]
 [ 0.00178712 -0.0261467  -0.03803866  0.10134279]]>, <Variable path=simple_dense_3/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>]
non_trainable_weights: []


ما در اینجا ها شیپ ورودی را میدانستیم ولی بعضی  جاها ممکنه که ندونیم که شیپ ورودی ما چیه اصا که خود ترنسور فلو هندلش میکنه و فقط نیازه شیپ خروجی بهش بدیم

In [88]:
class SimpleDense(tf.keras.layers.Layer):
    def  __init__(self,unit = 32):
        super(SimpleDense,self).__init__()
        self.unit = unit

    def build (self , input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1],self.unit),
            initializer=tf.random_normal_initializer,
            #initializer="random_normal",
            name='wight',
            trainable= True,
            dtype = 'float32'
        )
        self.b = self.add_weight(
            shape=(self.unit,),
            initializer='zeros',
            name='bias',
            trainable= True,
            dtype = 'float32'
        )
        super().build(input_shape)
        
    def call (self,inputs):
        return tf.matmul(inputs,self.w)+self.b

In [107]:
dense_layer = SimpleDense(32)

In [109]:
dense_layer.weights

[]

In [111]:
x = tf.ones((2, 2))
dense_layer = SimpleDense(32)
y = dense_layer(x)
print(y)

tf.Tensor(
[[ 0.03486611 -0.06377527  0.00716451 -0.01636961 -0.02240392 -0.03323643
   0.09562376 -0.06231388  0.11689482  0.06000452 -0.05718188  0.12092711
  -0.0594105  -0.00750078 -0.03701942 -0.01969307  0.04170824 -0.00476234
  -0.03558128 -0.13796332 -0.19467437  0.05485544 -0.01636355  0.07381389
   0.10517168 -0.01674053  0.06526749 -0.08120375  0.01263336 -0.05521955
  -0.07267843  0.06800267]
 [ 0.03486611 -0.06377527  0.00716451 -0.01636961 -0.02240392 -0.03323643
   0.09562376 -0.06231388  0.11689482  0.06000452 -0.05718188  0.12092711
  -0.0594105  -0.00750078 -0.03701942 -0.01969307  0.04170824 -0.00476234
  -0.03558128 -0.13796332 -0.19467437  0.05485544 -0.01636355  0.07381389
   0.10517168 -0.01674053  0.06526749 -0.08120375  0.01263336 -0.05521955
  -0.07267843  0.06800267]], shape=(2, 32), dtype=float32)


In [113]:
dense_layer.weights

[<Variable path=simple_dense_8/wight, shape=(2, 32), dtype=float32, value=[[ 0.03340961 -0.03246141  0.01200753  0.00107119 -0.04047243 -0.03453011
    0.10957509 -0.02005094  0.13657099  0.04901386  0.02708222  0.02856764
   -0.02332242  0.01568507  0.00557197 -0.00850939  0.0082498  -0.03835966
   -0.01009791 -0.08947428 -0.06905016  0.01343087 -0.00042211  0.037888
    0.02345038  0.01429988  0.05301615 -0.0940977   0.02793062 -0.0301109
   -0.06408771 -0.02698498]
  [ 0.0014565  -0.03131386 -0.00484302 -0.0174408   0.01806851  0.00129369
   -0.01395133 -0.04226294 -0.01967617  0.01099066 -0.0842641   0.09235948
   -0.03608808 -0.02318585 -0.04259139 -0.01118368  0.03345845  0.03359732
   -0.02548337 -0.04848905 -0.12562421  0.04142457 -0.01594144  0.03592589
    0.0817213  -0.03104041  0.01225133  0.01289394 -0.01529725 -0.02510864
   -0.00859072  0.09498765]]>,
 <Variable path=simple_dense_8/bias, shape=(32,), dtype=float32, value=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

## حالا ما می خوایم چندتا لایه را پکیج کنیم و توی یک لایه پرسنت کنیم

حود تنسور فلو پیشنهاد میده که اگه می خواهیم یک سری لایه را توی یک لایه تعریف کنیم توی همون اینیت بنویس نه بیلد

In [159]:
class multi_layer (tf.keras.layers.Layer):
    def __init__(self):
        super(multi_layer,self).__init__()
        self.linear1 = tf.keras.layers.Dense(32)
        self.linear2 = tf.keras.layers.Dense(32)
        self.linear3 = SimpleDense(2)

    def call(self,inputs):
        x = self.linear1(inputs)
        x = tf.keras.activations.relu(x)
        x = self.linear2(x)
        x = tf.nn.relu(x)
        return self.linear3(x)
        

In [161]:
mlp = multi_layer()
y = mlp(tf.ones(shape=(3, 64)))  # The first call to the `mlp` will create the weights
print("weights:", len(mlp.weights))
print("trainable weights:", len(mlp.trainable_weights))

weights: 6
trainable weights: 6


#  استفاده از لایه‌ی multi_layer در Functional API

In [168]:
inputs = tf.keras.Input(shape=(8,))
x = multi_layer()(inputs)
outputs = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ multi_layer_7 (multi_layer)     │ (None, 2)              │         1,410 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,413 (5.52 KB)

 Trainable params: 1,413 (5.52 KB)

 Non-trainable params: 0 (0.00 B)

## استفاده از multi_layer در Sequential API:

In [171]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(8,)),
    multi_layer(),
    tf.keras.layers.Dense(1)
])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ multi_layer_8 (multi_layer)     │ (None, 2)              │         1,410 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,413 (5.52 KB)

 Trainable params: 1,413 (5.52 KB)

 Non-trainable params: 0 (0.00 B)

بعضی لایه ها توی ترینینگ و تست باید رفتار متفاوتی نشان بدهند مثل لایه زیر یا بچ ها

In [16]:
class drop_me (tf.keras.layers.Layer):
    def __init__ (self, rate , **kwargs):
        super(drop_me , self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs_ , training__=None):
        if training__:
            return tf.nn.dropout(inputs_,rate=self.rate)
        else:
            return inputs_

In [18]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(8,)),
    drop_me(0.3),
    tf.keras.layers.Dense(1)
])
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ drop_me_2 (drop_me)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9 (36.00 B)

 Trainable params: 9 (36.00 B)

 Non-trainable params: 0 (0.00 B)